In [ ]:
# generate skip-grams
skip_grams = [tf.keras.preprocessing.sequence.skipgrams(wid, vocabulary_size=vocab_size, window_size=10) for wid in word_ids]
# view sample skip-grams
pairs, labels = skip_grams[0][0], skip_grams[0][1]
for i in range(10):
    print("({:s} ({:d}), {:s} ({:d})) -> {:d}".format(
          id_to_word[pairs[i][0]], pairs[i][0], 
          id_to_word[pairs[i][1]], pairs[i][1], 
          labels[i]))

In [ ]:
# generate skip-grams
skip_grams = [tf.keras.preprocessing.sequence.skipgrams(wid, vocabulary_size=vocab_size, window_size=10) for wid in word_ids]
# view sample skip-grams
pairs, labels = skip_grams[0][0], skip_grams[0][1]
for i in range(10):
    print("({:s} ({:d}), {:s} ({:d})) -> {:d}".format(
          id_to_word[pairs[i][0]], pairs[i][0], 
          id_to_word[pairs[i][1]], pairs[i][1], 
          labels[i]))



In [ ]:
from tensorflow.keras.layers import Concatenate, Dense, Embedding, Reshape
from tensorflow.keras.models import Model

# Define the input layers for the target and context words
target_word_input = tf.keras.Input(shape=(1,))
context_word_input = tf.keras.Input(shape=(1,))

# Build skip-gram architecture
target_word_model = Embedding(vocab_size, embedding_size,
                              embeddings_initializer="glorot_uniform")(target_word_input)
target_word_model = Reshape((embedding_size,))(target_word_model)

context_word_model = Embedding(vocab_size, embedding_size,
                               embeddings_initializer="glorot_uniform")(context_word_input)
context_word_model = Reshape((embedding_size,))(context_word_model)

# Concatenate the output of the target and context models
merged = Concatenate(axis=1)([target_word_model, context_word_model])

# Add a dense layer and sigmoid activation
output = Dense(1, kernel_initializer="glorot_uniform", activation="sigmoid")(merged)

# Define the model
model = Model(inputs=[target_word_input, context_word_input], outputs=output)

# Compile the model
model.compile(loss="mean_squared_error", optimizer="adam")

# View model summary
print(model.summary())


In [ ]:
# train the model on the skip-grams
for epoch in range(1, 6):
    total_loss = 0
    for i, elem in enumerate(skip_grams):
        skip_first_elem = np.array(list(zip(*elem[0]))[0], dtype='int32')
        skip_second_elem = np.array(list(zip(*elem[0]))[1], dtype='int32')
        labels = np.array(elem[1], dtype='int32')
        X = [skip_first_elem, skip_second_elem]
        Y = labels
        if i % 10000 == 0:
            print('Processed {} skip-gram pairs'.format(i))
        total_loss += model.train_on_batch(X,Y)  

    print('Epoch: {} Loss: {}'.format(epoch, total_loss))


In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import euclidean_distances

# get the embeddings for the words in the vocabulary
weights = model.layers[2].get_weights()[0]

distance_matrix = euclidean_distances(weights)
print(distance_matrix.shape)

similar_words = {search_term: [id2word[idx] for idx in distance_matrix[word2id[search_term]-1].argsort()[1:6]+1] 
                   for search_term in ['god', 'jesus', 'noah', 'egypt', 'john', 'gospel', 'moses','famine']}

print(similar_words)

# reduce the dimensions of the embeddings using t-SNE
tsne = TSNE(n_components=2, random_state=0)
vectors_2d = tsne.fit_transform(weights)

# create a list of the words in the vocabulary
words = [id2word[i] for i in range(1, vocab_size)]

# plot the similar words
fig, ax = plt.subplots(figsize=(20,10))
for word in similar_words:
    ax.scatter(vectors_2d[word2id[word]-1, 0], vectors_2d[word2id[word]-1, 1], c='red', label=word)
    for sim_word in similar_words[word]:
        ax.scatter(vectors_2d[word2id[sim_word]-1, 0], vectors_2d[word2id[sim_word]-1, 1], c='blue')
        ax.annotate(sim_word, (vectors_2d[word2id[sim_word]-1, 0], vectors_2d[word2id[sim_word]-1, 1]))
ax.legend()
plt.show()

